In [36]:

import numpy as np
from matplotlib import pyplot as plt
import ipywidgets as widgets

tof = np.load("30cmTrial1.npy")

def gating(x):
    var = np.var(x)
    mean = np.mean(x)
    x[x > mean + np.sqrt(var)] = mean
    return x

def feedbackFilter(x, a0, b1):
    y = np.zeros_like(x)
    y[0] = x[0]
    for i in range(1, len(x)):
        y[i] = a0 * x[i] + b1 * y[i-1]
    return y

def secondOrderFilter(x, a0, a1, b1, b2):
    y = np.zeros_like(x)
    y[0] = x[0]
    y[1] = a0 * x[1] + a1 * x[0] + b1 * y[0]
    for i in range(2, len(x)):
        y[i] = a0 * x[i] + a1 * x[i-1] + b1 * y[i-1] + b2 * y[i-2]
    return y



@widgets.interact(b1=widgets.FloatSlider(min=-1.0, max=1.0, step=0.01, value=0.95))
def execute(b1=0.95):
    x = gating(tof[6,3,:])
    filtered_tof = feedbackFilter(x, 1.0 - b1, b1)
    a1 = 0.1
    b2 = 0.1
    filtered_tof_2 = secondOrderFilter(x, 1.0 - b1 - a1, a1, b1 - b2, b2)
    
    fig, axes = plt.subplots(1, 2, figsize=(16,6))
    ax = axes[0]
    ax.plot(np.arange(tof.shape[2]), x, label="Channel 0")
    ax.plot(np.arange(tof.shape[2]), filtered_tof, label="Channel 1")
    ax.plot(np.arange(tof.shape[2]), filtered_tof_2, label="Channel 1")
    ax.plot(np.mean(x), label="Mean")

    plt.show



interactive(children=(FloatSlider(value=0.95, description='b1', max=1.0, min=-1.0, step=0.01), Output()), _dom…